<a href="https://colab.research.google.com/github/prathwishshetty/DS5500_hw4/blob/master/HW4_DS5500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h2>DS 5500 Homework 4</h2></center>


#####**Instructions**
Create a new public Github repository for this homework assignment. The repository should include all of
the code necessary to reproduce your submitted solutions. Do not include the raw data in the repository.


*Use the README.md of the Github repository to present your solutions. Answer all questions completely for
full credit, including figures and tables where appropriate. For each problem, either provide relevant inline
code snippets, or cite the source file where the relevant code lives (with line numbers if appropriate).*

Describe any data processing steps (transformation, filtering, etc.) you perform when solving each problem,
providing reasoning where appropriate. You may need to be creative when deciding how to approach each
problem, as there may not be a single “correct” solution.

Your solutions should be posted as a **public** note on Piazza in the hw4 folder with the title “[hw4] - your
name name”. Include in the body of the note a link to the Github repository with your solutions.

#####**Overview**

This homework assignment continues to use school data from the previous assignment.

Download the 2015-16 district-level fiscal data from the National Center for Education Statistics’ Common
Core of Data:
* https://nces.ed.gov/ccd/f33agency.asp

The 2015-16 performance statistics on graduation rate and state assessments on mathematics and reading/language arts are available from the EDFacts website:
* https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html

Additionally, the 2015-16 district-level “universe” survey data provides statistics about enrollment disaggregated by demographics like gender, race, disability status, etc:
* https://nces.ed.gov/ccd/pubagency.asp

These datasets can be linked based on the LEA IDs.

In [0]:
%%capture
!wget https://nces.ed.gov/ccd/data/zip/Sdf16_1a.zip
!unzip Sdf16_1a.zip
!wget https://www2.ed.gov/about/inits/ed/edfacts/data-files/math-achievement-sch-sy2015-16.csv
!wget https://www2.ed.gov/about/inits/ed/edfacts/data-files/rla-achievement-sch-sy2015-16.csv

!wget https://nces.ed.gov/ccd/Data/zip/ccd_lea_052_1516_w_1a_011717_csv.zip
!unzip ccd_lea_052_1516_w_1a_011717_csv.zip
!rm ccd_lea_052_1516_w_1a_011717_csv.zip

!wget https://nces.ed.gov/ccd/Data/zip/ccd_lea_002089_1516_w_1a_011717_csv.zip
!unzip ccd_lea_002089_1516_w_1a_011717_csv.zip
!rm ccd_lea_002089_1516_w_1a_011717_csv.zip

In [0]:
import pandas as pd 
import numpy as np 
import re 

data = pd.read_csv('Sdf16_1a.txt',sep='\t',low_memory=False)
math = pd.read_csv('math-achievement-sch-sy2015-16.csv',error_bad_lines=False, index_col=False,low_memory=False)
rla  = pd.read_csv('rla-achievement-sch-sy2015-16.csv',error_bad_lines=False, index_col=False,low_memory=False)
demo = pd.read_csv('ccd_lea_052_1516_w_1a_011717.csv',error_bad_lines=False, index_col=False,low_memory=False)
disb = pd.read_csv('ccd_lea_002089_1516_w_1a_011717.csv',error_bad_lines=False, index_col=False,low_memory=False)


In [0]:
#@title Function ColumnFixer
def columnFixer(data,column):

    data['temp_for_PS'] = data[column].fillna('PS')
    colVals = data.temp_for_PS.copy()

    def stringFixer(string):
        if string.isdigit()==True:x=int(string)
        elif "-" in string: x=int(np.mean([int(x) for x in string.split('-')]))
        elif string=='PS': x=np.nan
        elif string.isalnum()==True:
            alpha = re.findall("[a-zA-Z]+",string)[0]
            num   = int(re.findall(r'\d+', string)[0]) 
            if alpha == "LT":   x= int((0+(num-1))/2)
            elif alpha == "LE": x= int((0+num)/2)
            elif alpha == "GT": x= int(((num+1)+100)/2)
            elif alpha == "GE": x= int((num+100)/2)
        else:
            x = np.nan
        return x

    colVals_proc = [stringFixer(x) for x in colVals]

    col_name = column+'_proc'

    data[col_name]= colVals_proc
    data[col_name] =  data[col_name].fillna(np.nanmean(colVals_proc, axis=0))
    return data

#####**Problem 1**
For the districts you selected for budget cuts in HW 3 Problem 4, calculate and visualize the proportion of
each district’s total funding that will be lost.


In [0]:
pd.options.mode.chained_assignment = None
math_sub = math[['LEAID','ALL_MTH00PCTPROF_1516']].copy()
math_sub = columnFixer(math_sub,'ALL_MTH00PCTPROF_1516')
math_sub_grouped = math_sub.groupby(['LEAID'])['ALL_MTH00PCTPROF_1516_proc'].mean().reset_index()
rla_sub  =  rla[['LEAID','ALL_RLA00PCTPROF_1516']].copy()
rla_sub = columnFixer(rla_sub,'ALL_RLA00PCTPROF_1516')
rla_sub_grouped = rla_sub.groupby(['LEAID'])['ALL_RLA00PCTPROF_1516_proc'].mean().reset_index()

math_sub_grouped['LEAID'] = math_sub_grouped['LEAID'].astype(str)
data['LEAID'] = data['LEAID'].astype(str)
rla_sub_grouped['LEAID'] = rla_sub_grouped['LEAID'].astype(str)

data_math     = pd.merge(data,math_sub_grouped, on='LEAID', how='left')

data_math_rla = pd.merge(data_math,rla_sub_grouped,  on='LEAID',how='left') 
data_math_rla = data_math_rla[data_math_rla['ALL_MTH00PCTPROF_1516_proc'].notnull()]
data_math_rla = data_math_rla[data_math_rla['ALL_RLA00PCTPROF_1516_proc'].notnull()]
data_debt_grouped = data_math_rla[['LEAID','NAME','STNAME','TFEDREV','ALL_MTH00PCTPROF_1516_proc','ALL_RLA00PCTPROF_1516_proc']]
data_debt_grouped['Avg_Score'] = (data_debt_grouped['ALL_MTH00PCTPROF_1516_proc'] + data_debt_grouped['ALL_RLA00PCTPROF_1516_proc'])/2
#funding cut 1

cut1 = data_debt_grouped.query('Avg_Score>75')
cut1['fed_cut'] = cut1['TFEDREV'].multiply(0.30)
cut1['cut_prop'] = cut1['fed_cut']/cut1['TFEDREV']
cut1 = cut1[['LEAID','NAME','STNAME','TFEDREV','fed_cut','cut_prop']].copy()

cut2 = data_debt_grouped.query('Avg_Score>50 and Avg_Score<=75')
cut2['fed_cut'] = cut2['TFEDREV'].multiply(0.25)
cut2['cut_prop'] = cut2['fed_cut']/cut2['TFEDREV']
cut2 = cut2[['LEAID','NAME','STNAME','TFEDREV','fed_cut','cut_prop']].copy()


cut3 = data_debt_grouped.query('Avg_Score>25 and Avg_Score<=50')
cut3['fed_cut'] = cut3['TFEDREV'].multiply(0.15)
cut3['cut_prop'] = cut3['fed_cut']/cut3['TFEDREV']
cut3 = cut3[['LEAID','NAME','STNAME','TFEDREV','fed_cut','cut_prop']].copy()

cut4 = data_debt_grouped.query('Avg_Score<=25')
cut4['fed_cut'] = cut4['TFEDREV'].multiply(0)
cut4['cut_prop'] = cut4['fed_cut']/cut4['TFEDREV']
cut4 = cut4[['LEAID','NAME','STNAME','TFEDREV','fed_cut','cut_prop']].copy()

main_cut = pd.concat([cut1,cut2,cut3,cut4])
# main_cut.sort_values(by=['fed_cut'],ascending=False).head(10)

In [0]:
main_cut_agg = main_cut.groupby(['STNAME'])['TFEDREV','fed_cut'].agg('sum')

In [0]:
main_cut_agg['cut_prop'] = main_cut_agg['fed_cut']/main_cut_agg['TFEDREV']
main_cut_agg = main_cut_agg.reset_index()

In [8]:
import plotly.graph_objects as go

from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Fed Cut',  x=main_cut_agg['STNAME'], y=main_cut_agg['fed_cut']))
fig.add_trace(go.Bar(name='Revenue',  x=main_cut_agg['STNAME'], y=main_cut_agg['TFEDREV']))
fig.add_trace(go.Scatter(name='Cut Pecentage',x=main_cut_agg['STNAME'], y=main_cut_agg['cut_prop'],mode='lines'),secondary_y=True)

# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

Which districts will be affected by your budget cuts the most?

In [22]:
main_cut.query('cut_prop == 0.3')['NAME']

2880      Charter School of Wilmington
2885                    Sussex Academy
2887             Newark Charter School
2891                MOT Charter School
3078                       FAU LAB SCH
                     ...              
17952    Mercer Island School District
18054         St. John School District
18175     Lake Country School District
18450          Swallow School District
18451       North Lake School District
Name: NAME, Length: 1793, dtype: object

#####**Problem 2**

A common problem with purely data-driven solutions is that they can inadvertently perpetuate hidden
pre-existing biases in the data, and further disadvantage groups that are already disadvantaged.

Calculate the proportion of enrolled students by race for each district, then visualize the distributions of
these for districts that received budget cuts versus districts that did not receive budget cuts.

In [0]:
#@title
cols = ['AM','AS','HI','BL','WH','HP','TR','TOTAL']
demo_race = demo[['LEAID']+cols]

main_cut['LEAID'] = main_cut['LEAID'].astype(str)
demo_race['LEAID'] = demo_race['LEAID'].astype(str)

main_cut_demo = main_cut.merge(demo_race[['LEAID']+cols].query('TOTAL >= 0'),
                               on='LEAID',
                               how='left',
                               indicator=True)

main_cut_demo['race_0_1'] = ['no fed cut' if x == 0.00 else 'fed cut' for x in main_cut_demo['cut_prop']]

main_cut_demo_agg = main_cut_demo.groupby(['cut_prop'])[cols].agg('sum')
main_cut_demo_agg['calcTotal'] =  (main_cut_demo_agg[['AM','AS','HI','BL','WH','HP','TR']].sum(axis=1))

main_cut_demo_agg = main_cut_demo_agg.reset_index()
main_cut_demo_agg['cut_prop'] = main_cut_demo_agg['cut_prop'].astype(str)

In [0]:
#@title
main_cut_fed_demo_agg = main_cut_demo.groupby(['cut_prop'])[cols].agg('sum')
main_cut_fed_demo_agg['calcTotal'] =  (main_cut_fed_demo_agg[['AM','AS','HI','BL','WH','HP','TR']].sum(axis=1))

In [0]:
main_cut_fed_demo_agg['tot_all'] = main_cut_fed_demo_agg['calcTotal'].sum()

In [54]:
import plotly.graph_objects as go

x=['cut of 0.00', 'cut of 0.25 ', 'cut of 0.25', 'cut of 0.3']
data = main_cut_fed_demo_agg.copy()
fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=(data['AM']/data['calcTotal'])*100, name='American Indian/Alaska Native'))
fig.add_trace(go.Bar(x=x, y=(data['HP']/data['calcTotal'])*100, name='Hawaiian Native / Pacific Islander'))
fig.add_trace(go.Bar(x=x, y=(data['TR']/data['calcTotal'])*100, name='Two or More Races'))
fig.add_trace(go.Bar(x=x, y=(data['AS']/data['calcTotal'])*100, name='Asian'))
fig.add_trace(go.Bar(x=x, y=(data['HI']/data['calcTotal'])*100, name='Hispanic'))
fig.add_trace(go.Bar(x=x, y=(data['BL']/data['calcTotal'])*100, name='Black'))
fig.add_trace(go.Bar(x=x, y=(data['WH']/data['calcTotal'])*100, name='White'))

fig.update_layout(barmode='stack', 
                  xaxis={'categoryorder':'category ascending'},
                  title='Fed cut per race')
fig.show()


Comment on whether the the distributions appear to be the same or different. Did your selection include any
hidden biases, or manage to avoid them?

#####**Solution**

The distribution does look different. We see that `white` category are the ones are the one most affected by the fed cut. And `black` category are the one's relatively less affected by the cut in funds. While hispanic community has equal distribution.

#####**Problem 3**
Calculate the proportion of enrolled students by disability status (students with an IEP under IDEA) for each
district, then visualize the distributions of these proportions for districts that received budget cuts versus
districts that did not receive budget cuts.

In [0]:
#@title
demo_disb = pd.merge(disb, demo[['LEAID', 'TOTAL']], on='LEAID', how='inner')

cols = ['SPECED','TOTAL']
disb_lvl = demo_disb[['LEAID']+cols]

main_cut['LEAID'] = main_cut['LEAID'].astype(str)
disb_lvl['LEAID'] = demo_race['LEAID'].astype(str)

main_cut_disb = main_cut.merge(disb_lvl[['LEAID']+cols],
                               on='LEAID',
                               how='left',
                               indicator=True).query('TOTAL >= 0 and SPECED>0')

main_disb_agg = main_cut_disb.groupby(['STNAME'])['SPECED','TOTAL'].agg('sum').reset_index()
main_disb_agg['Proportion'] = main_disb_agg['SPECED']/main_disb_agg['TOTAL']

In [63]:
#@title
import plotly.graph_objects as go

from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Disability',  x=main_disb_agg['STNAME'], y=main_disb_agg['SPECED']))
fig.add_trace(go.Bar(name='Total',  x=main_disb_agg['STNAME'], y=main_disb_agg['TOTAL']))
fig.add_trace(go.Scatter(name='Pecentage of Disbabled',x=main_disb_agg['STNAME'], y=main_disb_agg['Proportion'],mode='lines'),secondary_y=True)

# Change the bar mode
fig.update_layout(barmode='stack')

# Add figure title
fig.update_layout(title_text="Disability Status",width=1000, height=500)

# Set x-axis title
fig.update_xaxes(title_text="States")

# Set y-axes titles
fig.update_yaxes(title_text="Number of students", secondary_y=False)
fig.update_yaxes(title_text="Percentage with disability", secondary_y=True)

fig.show()

In [0]:
#@title
main_cut_disb['disb_0_1'] = ['fed_cut' if x == 0.00 else 'no fed_cut' for x in main_cut_disb['cut_prop']]
main_cut_disb_agg = main_cut_disb.groupby(['disb_0_1'])[cols].agg('sum').reset_index()
main_cut_disb_agg['tot_sum']= main_cut_disb_agg["TOTAL"].sum()

In [65]:
#@title
import plotly.graph_objects as go

# x=['cut of 0', 'cut of 0.15', 'cut of 0.25', 'cut of 0.3']

data = main_cut_disb_agg.copy()
fig = go.Figure()
fig.add_trace(go.Bar(x=data['disb_0_1'], y=(data['SPECED']/data['SPECED'].sum())))

# Add figure title
fig.update_layout(title_text="Percentage with Disability Status per Fed Cut")

# Set x-axis title
fig.update_xaxes(title_text="Percentage of federal cut")

# Set y-axes titles
fig.update_yaxes(title_text="Percentage of disabled students")


fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'},width=600, height=400)
fig.show()

Comment on whether the the distributions appear to be the same or different. Did your selection include any
hidden biases, or manage to avoid them?

######**Solution**
Disabled are the least affected in the fed cut. The reason for the same is that i have used performance to cut funds and usually disabled students tend to struggle in the normal schooling system. 
Hence, I did not cut their funding at all

#####**Problem 4**

Choose and critique one of your fellow classmates’ selection of schools for budget cuts in HW 3 Problem 4 and Problem 5. What was the justification of their selection? Discuss any advantages or disadvantages of their approach.

######**Solution:**
I chose to critique Rutu's HW3 solution for this problem.

She has chosen to cut revenues of school's that make more money than they spend or school's whose dependnce on federal funds is less than 80%.

This could be a good solution as these schools on the outset don't need as much money as other schools who are in debt.

But this doesn't take into account the schools performance on the various tests i.e. if students are performing badly at these schools cutting funding wouldn't be a good idea. Also, punishing schools that reduce expendture is not a good idea, as it could encourage schools to give inflated expenditure numbers.

#####**Problem 5**
Summarize and comment on what you learned from one the special topics lectures (MapReduce + Hadoop, Visualization, Causal Inference, or the Industry Panel) of your choice.


######**Solution**

I have decided to summarize the Industry Panel which included the following panelists:
* Faye Zheng of *Gamalon*
* Michelle Tat of *Haven Healthcare*
* Daniel Hannah *Vectra AI*
* Katie Porter *Tamr*
* Luke Winslow of *Wayfair* 

The panelists described the various experiences they had in the industry as a Data Scientist. The panelists talked about the diversity in the title of various Data Science positions and in their naming. This indicated how the hiring process is still in its very nascency as the panelists came to a consesus that the job requirements is more  like a 'good to have' and not really a requirement.

In the next panel i would like to see a little more diversity in the roles played by the panelists i.e. maybe a techincal recruiter or a data scientist who is working on development of models(algorithms) or someone who has come from educational background in computer science to whom we can relate.